# Organics-Spend Exploratory Analysis

Exploratory analysis to understand the dependence of Organic Signups on spend at the channel level. The 3 markets MX, PH, and KE were explored. In each case, data included the number of organic signups on a weekly frequency as well as the corresponding google and facebook spend. The analysis is broken down individually to each market.

---

In [63]:
import numpy as np
import pandas as pd
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import optimize, curve_fit
from scipy.stats import linregress

import plotly.io as pio

# change default plotly theme
pio.templates.default = "plotly_white"

In [129]:
# load datasets
mx_data = pd.read_csv('data/mx_organic_spend.csv', header=1)
ph_data = pd.read_csv('data/ph_organic_spend.csv', header=1)
ke_data = pd.read_csv('data/ke_organic_spend.csv', header=1)

In [130]:
def clean(data, market=None):
    
    # drop unneeded cols and rows
    if market=='MX':
        keep_cols = ['Week of ', 'Total Signups', 'Organic Signups', 
             'MX_GoogleUAC_tCPA-high_KYC-es-MX', 'MX_7/14/21_Broad_KYC - High']
        data = data.loc[:28, keep_cols]
        data = data.rename(columns={'MX_GoogleUAC_tCPA-high_KYC-es-MX': 'google_spend',
                    'MX_7/14/21_Broad_KYC - High': 'fb_spend'})
        
        data.fb_spend = data.fb_spend.str.replace('[$,]', '', regex=True).astype(float)
        
    elif market=='PH':
        keep_cols = ['Week of ', 'Total Signups', 'Organic Signups', 
             'PH_GoogleUAC_tCPA-high_KYC-en-PH', 'PH - KYC Approval - High-Score - 1D Click - NO BID']
        data = data.loc[:27, keep_cols]
        data = data.rename(columns={'PH_GoogleUAC_tCPA-high_KYC-en-PH': 'google_spend',
                    'PH - KYC Approval - High-Score - 1D Click - NO BID': 'fb_spend'})
        
        data.fb_spend = data.fb_spend.str.replace('[$,]', '', regex=True).astype(float)
        data.google_spend = data['google_spend'].str.replace('[$,]', '', regex=True).astype(float)
        
    elif market=='KE':
        keep_cols = ['Week of ', 'Total Signups', 'Organic Signups', 
             'KE_GoogleUAC_tCPA-high_KYC-en-KE', 'Tala - KE - Android']
        data = data.loc[:28, keep_cols]
        data = data.rename(columns={'KE_GoogleUAC_tCPA-high_KYC-en-KE': 'google_spend',
                    'Tala - KE - Android': 'fb_spend'})
        
        data.fb_spend = data.fb_spend.str.replace('[$,]', '', regex=True).astype(float)
        data.google_spend = data['google_spend'].str.replace('[$,]', '', regex=True).astype(float)
    
    return data

In [131]:
mx_data = clean(mx_data, market='MX')
ph_data = clean(ph_data, market='PH')
ke_data = clean(ke_data, market='KE')

In [179]:
def plot_series(data, fields, normalize=True):
    traces=[]
    for f in fields:
        if normalize:
            traces.append(go.Scatter(name=f, x=data['Week of '], y=data[f]/data[f].max()))
        else:
            traces.append(go.Scatter(name=f, x=data['Week of '], y=data[f]))
    
    fig = go.Figure(traces)
    
    if normalize:
        fig.update_layout(xaxis=dict(title='Week of'), yaxis=dict(title='Normalized Value'))
    else:
        fig.update_layout(xaxis=dict(title='Week of'), yaxis=dict(title='Value'))
    
    return fig

## MX Analysis

In [180]:
fig = plot_series(mx_data, ['Organic Signups', 'google_spend', 'fb_spend'], normalize=True)
fig.show()

The plot above shows *Organic signups*, *Google spend*, and *Facebook spend*. Each curve has been normalized by dividing it by its max value. This effectively scales all curves to between 0 and 1 so that we can compare their features on the same scale.

Comparing the *signups* curve to the *spend* curves, there appears to be some correlation between *signups* and *fb_spend*, but less so with *google_spend*. In the beginning, all 3 curves increase until around Nov, where both google and fb spend are dramatically decreased. 

In [37]:
fig = go.Figure(
    go.Scatter(y=mx_data['Organic Signups'], x=mx_data['google_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='Google Spend'))

fig.show()

In [38]:
fig = go.Figure(
    go.Scatter(y=mx_data['Organic Signups'], x=mx_data['fb_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

In [52]:
fig = go.Figure(data=go.Histogram(x=mx_data['Organic Signups'], xbins=dict(start=7000, end=16000, size=1000)))
fig.update_layout(title='Organic Signups Distribution')
fig.show()

In [56]:
fig = go.Figure(data=go.Histogram(x=mx_data['google_spend'], xbins=dict(start=13000, end=68000, size=10000)))
fig.update_layout(title='Google Spend Distribution')
fig.show()

In [58]:
fig = go.Figure(data=go.Histogram(x=mx_data['fb_spend'], xbins=dict(start=400, end=23000, size=1000)))
fig.update_layout(title='Facebook Spend Distribution')
fig.show()

In [59]:
mx_data[['Organic Signups', 'google_spend', 'fb_spend']].cov().iloc[0]

Organic Signups    4.442689e+06
google_spend       1.896917e+06
fb_spend           1.082936e+07
Name: Organic Signups, dtype: float64

In [60]:
mx_data[['Organic Signups', 'google_spend', 'fb_spend']].corr().iloc[0]

Organic Signups    1.000000
google_spend       0.061167
fb_spend           0.754860
Name: Organic Signups, dtype: float64

In [76]:
res = linregress(mx_data[['fb_spend', 'Organic Signups']])

In [79]:
fig = go.Figure([
    go.Scatter(name='data', y=mx_data['Organic Signups'], x=mx_data['fb_spend'], mode='markers',
              marker=dict(size=10)),
    go.Scatter(name='regression', y=(res.slope*mx_data['fb_spend'])+res.intercept, x=mx_data['fb_spend'])
])

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

In [83]:
mx_data['google_spend']/mx_data['fb_spend']

0      1.947111
1      2.479370
2      3.200026
3      3.197468
4      3.061763
5      3.386181
6      3.067164
7      2.677800
8      3.083615
9      8.058001
10     9.048222
11     4.210752
12     2.890625
13     7.193310
14    15.564065
15    24.136998
16    23.587909
17    34.099789
18    39.315045
19    37.519555
20    41.479727
21    36.945251
22    30.277847
23    30.337283
24    34.014967
25    28.409000
26    13.071174
27    10.071714
28    32.326416
dtype: float64

In [86]:
(mx_data['google_spend']/mx_data['fb_spend']).mean()

16.850280922783437

## PH

In [112]:
fig = plot_series(ph_data, ['Organic Signups', 'google_spend', 'fb_spend'])
fig.show()

In [114]:
fig = go.Figure(
    go.Scatter(y=ph_data['Organic Signups'], x=ph_data['google_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='Google Spend'))

fig.show()

In [116]:
fig = go.Figure(
    go.Scatter(y=ph_data['Organic Signups'], x=ph_data['fb_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

In [121]:
fig = go.Figure(data=go.Histogram(x=ph_data['Organic Signups'], xbins=dict(start=8000, end=45000, size=2000)))
fig.update_layout(title='Organic Signups Distribution')
fig.show()

In [136]:
ph_data[ph_data['Organic Signups'] < 12000]

,Week of,Total Signups,Organic Signups,google_spend,fb_spend
14,Dec 20,16483.0,8991,25811.0,5736.0
15,Dec 27,17244.0,8770,36296.0,5728.0
16,Jan 3,21301.0,11828,30449.0,4171.0


In [137]:
ph_data = ph_data[ph_data['Organic Signups']>12000]

In [138]:
fig = go.Figure(
    go.Scatter(y=ph_data['Organic Signups'], x=ph_data['google_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='Google Spend'))

fig.show()

In [139]:
fig = go.Figure(
    go.Scatter(y=ph_data['Organic Signups'], x=ph_data['fb_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

In [140]:
ph_data[['Organic Signups', 'google_spend', 'fb_spend']].cov().iloc[0]

Organic Signups    4.245765e+06
google_spend       7.133627e+06
fb_spend           5.592271e+05
Name: Organic Signups, dtype: float64

In [141]:
ph_data[['Organic Signups', 'google_spend', 'fb_spend']].corr().iloc[0]

Organic Signups    1.000000
google_spend       0.505006
fb_spend           0.140909
Name: Organic Signups, dtype: float64

In [142]:
res = linregress(ph_data[['google_spend', 'Organic Signups']])

In [143]:
fig = go.Figure([
    go.Scatter(name='data', y=ph_data['Organic Signups'], x=ph_data['google_spend'], mode='markers',
              marker=dict(size=10)),
    go.Scatter(name='regression', y=(res.slope*ph_data['google_spend'])+res.intercept, x=ph_data['google_spend'])
])

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

## KE

In [144]:
fig = plot_series(ke_data, ['Organic Signups', 'google_spend', 'fb_spend'])
fig.show()

In [145]:
fig = go.Figure(
    go.Scatter(y=ke_data['Organic Signups'], x=ke_data['google_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='Google Spend'))

fig.show()

In [146]:
fig = go.Figure(
    go.Scatter(y=ke_data['Organic Signups'], x=ke_data['fb_spend'], mode='markers',
              marker=dict(size=10))
)

fig.update_layout(yaxis=dict(title='Organic Signups'),
                 xaxis=dict(title='FB Spend'))

fig.show()

In [155]:
fig = go.Figure(data=go.Histogram(x=ke_data['Organic Signups'], xbins=dict(start=1000, end=16000, size=2000)))
fig.update_layout(title='Organic Signups Distribution')
fig.show()

In [158]:
ke_data[['Organic Signups', 'google_spend', 'fb_spend']].cov().iloc[0]

Organic Signups    1.684612e+06
google_spend      -9.812244e+05
fb_spend          -2.490547e+05
Name: Organic Signups, dtype: float64

In [159]:
ke_data[['Organic Signups', 'google_spend', 'fb_spend']].corr().iloc[0]

Organic Signups    1.000000
google_spend      -0.154317
fb_spend          -0.162579
Name: Organic Signups, dtype: float64

## Summary

1. Organic Signups 